In [1]:
import os
os.environ['DISPLAY'] = ':0.0'
import shutil
import numpy as np
from pathlib import Path
from GFG import Ctx
from GFG.model import Nf
from GFG.identity import IDModel
from GFG.model import Furmap, hcol
from ogbGL.draw_objects import Camera
from skimage import transform

np.random.seed(42)
try:
    ROOT = Path(__file__).parents[1]
except:
    ROOT = Path('/home/lukass/deep-gmf')

Made context-safe: Ebo bind <class 'function'>
Made context-safe: Ebo delete <class 'function'>
Made context-safe: Vbo bind <class 'function'>
Made context-safe: Vbo update <class 'function'>
Made context-safe: Vbo delete <class 'function'>
Made context-safe: Texture create_texture2D <class 'function'>
Made context-safe: Texture create_texture2DMS <class 'function'>
Made context-safe: Texture create_texture2Darr <class 'function'>
Made context-safe: Texture set_as_active <class 'function'>
Made context-safe: Texture delete <class 'function'>
Made context-safe: Renderbuffer create_renderbuffer <class 'function'>
Made context-safe: Renderbuffer create_renderbufferMS <class 'function'>
Made context-safe: Renderbuffer delete <class 'function'>
Made context-safe: Fbo bind <class 'function'>
Made context-safe: Fbo initialise <class 'function'>
Made context-safe: Fbo assert_complete <class 'function'>
Made context-safe: Fbo delete <class 'function'>
Made context-safe: ShaderProgram update_uni

In [2]:
# Load identity model
data_path = '/analyse/Project0294/GFG_data/model_HDB_linear_v2dense_compact.mat'
idm = IDModel.load(data_path)
base_nf = Nf.from_default()

# furmap = Furmap(hcol=hcol['Terra Cotta'], nlayers=120, density=1.)
# furmap.material.split_newmtl(mat_groups = base_nf.material_groups,
#                              fvt = base_nf.face_tuple.fv,
#                              vt = base_nf.vertex_tuple.dSvt,
#                              groupfidx = base_nf.groupfindex)
# base_nf.attach_fur(furmap, name='fur')

In [ ]:
# Setup openGL context + camera
ctx = Ctx(res=(256, 256), renderscale=4.)
ctx.camera[0] = Camera(
    ctx.win, ctx._ar,
    target = [-11.5644, -13.0381, 0],
    eye = [-11.5644, -13.0381, 700.],
    up = [0,1,0],
    FOV = 50,
    near = 100.0,
    far = 1000.0
)
ctx.transform_lights(0, 0, 0, order='xyz')

In [ ]:
age = 25
ethn = 'WC'
gender = 'M'
v_coeff = np.random.normal(0, 1, size=len(idm))
t_coeff = np.random.normal(0, 1, size=(idm.nbands, len(idm)))

tdet = np.load('/analyse/Project0257/lukas/data/tdet.npy')
nf = idm.generate(v_coeff, t_coeff, ethnicity=ethn, gender=gender, age=age,
                  basenf=base_nf, tdet=tdet)
nf.attach(ctx)

In [ ]:
ctx.set_lights('flat')
img = ctx.render('image')
img.save('flat.png')


In [ ]:
nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
nf.transform_model(15, 25, 0, [0, 0, 0], order='xyzt', replace=False)
img = ctx.render('image')
img.save('rot.png')

In [ ]:
nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
nf.transform_model(0, 0, 0, [0, 0, 0], order='xyzt', replace=False)
nf.transform_model(0, 0, 0, [0, 0, 100], order='xyzt', replace=False)

img = ctx.render('image')

img_arr = np.array(img)
tform = transform.EuclideanTransform(
   translation = (+0, 0)
   )
img_arr = transform.warp(img_arr, tform.inverse)
img_arr *= 255

img_arr[191:193, 126:128, :] = [255, 255, 255, 255]
img_arr[126:128, 126:128, :] = [255, 255, 255, 255]
img_arr[63:65, 126:128, :] = [255, 255, 255, 255]
img_arr[126:128, 191:193, :] = [255, 255, 255, 255]
img_arr[126:128, 126:128, :] = [255, 255, 255, 255]
img_arr[126:128, 63:65, :] = [255, 255, 255, 255]

img_arr[63:65, 63:65, :] = [255, 255, 255, 255]
img_arr[191:193, 63:65, :] = [255, 255, 255, 255]

img_arr[63:65, 191:193, :] = [255, 255, 255, 255]
img_arr[191:193, 191:193, :] = [255, 255, 255, 255]
img = Image.fromarray(img_arr.astype(np.uint8))

#img.save('test.png')
img

In [ ]:
from GFG.model import Adata
adata = Adata.load('quick_FACS_blendshapes_v2dense')
adata.attach(ctx)

In [ ]:
adata.labels_AU[10:]

In [ ]:
adata.bshapes['AU4'] = 0
adata.bshapes['AU9'] = 1
adata.bshapes['AU10Open'] = 1
adata.bshapes['AU16Open'] = 0
adata.bshapes['AU22'] = 0


In [ ]:
nf.transform_model(0, 0, 0, [0, 0, 0], order='txyz', replace=True)
nf.transform_model(0, 0, 0, [0, 0, 0], order='xyzt', replace=False)

ctx.set_lights(Path(ROOT / 'lights.yaml'))
ctx.transform_lights(90, 0, 45, [0, 0, 0])
img = ctx.render('image')
img

In [ ]:
from PIL import Image
img = ctx.render(dest='image')
img_arr = np.array(img)
img_rgb, img_a = img_arr[..., :3], img_arr[..., 3, None] / 255.
bg_rgb = np.array(Image.open(ROOT / 'data' / f'background_1.png'))
img_arr = (img_rgb * img_a) + (bg_rgb * (1 - img_a))  # alpha blend
img_arr = np.dstack((img_arr, np.ones((256, 256)) * 255)).astype(np.uint8)

img = Image.fromarray(img_arr)
img.save('background.png')

In [ ]:
from pathlib import Path
ctx.set_lights(Path('lights.yaml'))
#ctx.transform_lights(0, 0, 0, [0, 0, 0])

import imageio

with imageio.get_writer('lights.gif', mode='I', fps=60) as writer:
    #for i in range(360):
    #    ctx.transform_lights(1, 0, 0, [0, 0, 0])        
    #    img = ctx.render('image')
    #    writer.append_data(np.array(img))
        
    #for i in range(360):
    #    ctx.transform_lights(0, 1, 0, [0, 0, 0])        
    #    img = ctx.render('image')
    #    writer.append_data(np.array(img))
        
    for i in range(360):
        ctx.transform_lights(0, 0, 1, [0, 0, 0])        
        img = ctx.render('image')
        writer.append_data(np.array(img))
    

In [ ]:
#ctx.set_lights(Path('lights.yaml'))
#ctx.transform_lights(0, 10, 0, [0, 0, 0])
#base_nf.attach(ctx)
img = ctx.render('image')
img